# CNKI数据检索

按照用户输入的检索词，从知网上获得对应的文献信息，信息包括题目、关键词、作者、机构、发表时间、下载次数、被引次数。并可以将这些信息用于后续的数据分析。

为了方便爬取，我选择的入口地址是'http://search.cnki.com.cn/'。
用户输入检索词并点击搜索后，会跳转到另外一个页面，我们再对跳转后得到的页面的URL进行分析。
如输入检索词“人工智能”，得到的页面的URL为'http://search.cnki.com.cn/search.aspx?q=%e4%ba%ba%e5%b7%a5%e6%99%ba%e8%83%bd&rank=relevant&cluster=all&val=&p=0'
可以看出，其中的'q=%e4%ba%ba%e5%b7%a5%e6%99%ba%e8%83%bd'被编码了，我们可以用urllib.parse.unquote()进行解码，发现解码后的URL为'http://search.cnki.com.cn/search.aspx?q=人工智能&rank=relevant&cluster=all&val=&p=0'。
其中，q的值是用户输入的检索词；rank的值表明文献按哪种方式排序，默认按相关度，即'rank=relevant'，也可以按被引次数，为'rank=citeNumber'，也可以按下载次数，为'rank=download'，也可以按时间，为'date'；p的值用于控制翻页，p=0
为第1页，每一页有15条记录，故第2页为p=15，第3页为p=30，以此类推。

在掌握了获取URL的方法后，下面我们开始对网页上的数据进行爬取。用列表存储获得的数据，用字典的形式存储一条文献的信息。但是由于被引次数、下载次数和关键词、作者这两组信息分别存储在两个不同的网页中，
而题目、机构、发表时间在两个网页中都有，所以用两个列表，一个存储题目、机构、下载次数、被引次数，一个存储题目、关键词、机构、作者、发表时间，将两个列表转换为DataFrame形式，然后以题目和机构作为公共属性合并两个DataFrame。


下面先写出代码的基本框架：

In [ ]:
from urllib.parse import quote
import requests
from bs4 import BeautifulSoup
import pandas as pd

#获得页面上的HTML内容
def getHTMLText(url):
    return ''

#获得搜索到的页面上的各篇论文的详细信息的入口地址，把所有的入口地址存储在列表中
def getURLList(url,hrefList):
    pass

#获得论文的题目、作者、机构、发表时间、关键词
def getPaperInfo(lst,hrefList):
    pass

#获得论文的题目、机构、下载次数、被引次数
def getPaperInfo2(url,lst):
    pass

#将两个列表转换为DataFrame类型，并把两个列表连接起来
def formatDataset(lst1,lst2):
    pass

#定义main函数调用其他的函数
def main():
    quaryWord=input("请输入检索词:")
    pageNum=2
    paperInfo=[]    #用于存储论文的题目、作者、机构、发表时间、关键词
    paperInfo2=[]   #用于存储论文的题目、机构、下载次数、被引次数
    start_url='http://search.cnki.com.cn'+'/Search.aspx?q='+quote(quaryWord)+'&rank=relevant=&cluster=all&val='
    for i in range(pageNum):
        hrefs=[]
        quary_url=start_url+'&p='+str(i*15)
        getURLList(quary_url,hrefs)
        getPaperInfo2(quary_url,paperInfo2)
        getPaperInfo(paperInfo,hrefs)
    formatDataset(paperInfo,paperInfo2)
    
main()

在main函数中，pageNum表明从第一页开始一共需要爬取多少页，quote(quaryWord)用于对用户输入的检索词进行编码，quary_url为根据用户输入的检索词和页码生成的URL。

在getHTMLText函数中，我们可以用requests库获得html页面的源码

In [ ]:
def getHTMLText(url):
    try:
        r=requests.get(url)
        r.raise_for_status
        r.encoding='utf-8'
        return r.text
    except:
        return ''

举个例子，当用户输入“人工智能”时得到的页面的源码中，要找到其中第一篇论文的入口地址，需要找到的代码段为：

对于其余的14篇论文，其入口也是在这个相应的代码段中。在上面代码中可以看出，要找到入口地址，要先找到具有{'class':'wz_content'}属性的div标签，再在其中找到第一个具有{'target':_blank'}属性的a标签，获取其中的href属性的值。代码如下：

In [ ]:
def getURLList(url,hrefList):
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    for div in soup.find_all('div',attrs={'class':'wz_content'}):
        try:
            hrefList.append(div.find('a',attrs={'target':'_blank'}).get('href'))
        except:
            continue

获取论文的题目、机构、下载次数、被引次数时，也是从上面的html代码中获取信息，如题目为第一个a标签中的字符串内容；要找到机构，要先找到具有{'class':'year-count'}属性的span标签，再找到其中的第一个span标签，获得其中title属性的值；
下载次数和被引次数为具有{'class':'count'}属性的span标签中的字符串内容。代码如下：

In [ ]:
def getPaperInfo2(url,lst):
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')

    for div in soup.find_all('div',attrs={'class':'wz_content'}):
        infoDict={}
        try:
            infoDict.update({'题目':div.find('a',attrs={'target':'_blank'}).text})
            
            span=div.find('span',attrs={'class':'year-count'})
            infoDict.update({'发表机构':span.find('span').get('title')})
            
            for span in div.find_all('span',attrs={'class':'count'}):
                infoDict.update({'下载数':span.string.split('|')[0].replace('下载次数（','').replace('）','')})
                infoDict.update({'被引数':span.string.split('|')[1].replace('被引次数（','').replace('）','')})
                
            lst.append(infoDict)
        except:
            continue

在其中一篇论文的页面中，获取论文的题目、关键词时，可以选择从下面的html文段中获取信息。题目为title标签中的字符串内容，关键词为具有{'name':'keywords'}属性的meta标签中content属性的值。

获取作者姓名时，可以从下面的html代码段中获取信息。由于一篇论文的作者可能不止一人，因此可以用列表存储一篇文章的作者的姓名

获取机构和发表时间时，可以从下面的html代码中获取信息。其中b标签和font标枪均为在此页面的源码中第一次出现，因此可以用find()定位。

下面给出获取文献的题目、作者、机构、发表时间、关键词的函数的定义

In [ ]:
def getPaperInfo(lst,hrefList):
    for url in hrefList:
        infoDict={}
        try:
            html=getHTMLText(url)
            soup=BeautifulSoup(html,'html.parser')
            
            infoDict['题目']=soup.head.title.string.split('-')[0]
            
            infoDict['关键词']=soup.head.find_all('meta',attrs={'name':'keywords'})[0].get('content')
            
            authors=[]
            for div in soup.find_all('div',style="text-align:center; width:740px; height:30px;"):
                for a in div.find_all('a',target="_blank"):
                    authors.append(a.string)
            infoDict['作者']=authors
            
            infoDict['发表机构']=soup.find('b').string.strip().replace('《','').replace('》','')
            
            infoDict['发表时间']=soup.find('font').string.strip()
            
            lst.append(infoDict)
            
        except:
            continue

下面给出完整的代码实现

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

#获得输入url的页面内容
def getHTMLText(url):
    try:
        r=requests.get(url)
        r.raise_for_status
        r.encoding='utf-8'
        return r.text
    except:
        return ''

#获得在指定页数的查询页面中的查找到的论文的入口地址
def getURLList(url,hrefList):
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    for div in soup.find_all('div',attrs={'class':'wz_content'}):
        try:
            hrefList.append(div.find('a',attrs={'target':'_blank'}).get('href'))
        except:
            continue
            
#获得论文的被引数和下载数、题目、机构
def getPaperInfo2(url,lst):
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    for div in soup.find_all('div',attrs={'class':'wz_content'}):
        infoDict={}
        try:
            infoDict.update({'题目':div.find('a',attrs={'target':'_blank'}).text})
            
            span=div.find('span',attrs={'class':'year-count'})
            infoDict.update({'发表机构':span.find('span').get('title')})
            
            for span in div.find_all('span',attrs={'class':'count'}):
                infoDict.update({'下载数':span.string.split('|')[0].replace('下载次数（','').replace('）','')})
                infoDict.update({'被引数':span.string.split('|')[1].replace('被引次数（','').replace('）','')})
                
            lst.append(infoDict)
        except:
            continue
                      
#获得论文的题目、作者、机构、时间、关键词
def getPaperInfo(lst,hrefList):
    for url in hrefList:
        infoDict={}
        try:
            html=getHTMLText(url)
            soup=BeautifulSoup(html,'html.parser')
            
            infoDict['题目']=soup.head.title.string.split('-')[0]
            
            infoDict['关键词']=soup.head.find_all('meta',attrs={'name':'keywords'})[0].get('content')
            
            authors=[]
            for div in soup.find_all('div',style="text-align:center; width:740px; height:30px;"):
                for a in div.find_all('a',target="_blank"):
                    authors.append(a.string)
            infoDict['作者']=authors
            
            infoDict['发表机构']=soup.find('b').string.strip().replace('《','').replace('》','')
            
            infoDict['发表时间']=soup.find('font').string.strip()
            
            lst.append(infoDict)            
        except:
            continue

#用DataFrame把两个列表合并起来
def getFormatDataset(lst1,lst2):
    df1=pd.DataFrame(lst1)
    df2=pd.DataFrame(lst2)
    
    df=pd.merge(df1,df2,on=['题目','发表机构'])
    return df
    
def main():
    quaryWord=input("请输入搜索信息:")
    pageNum=2
    paperInfo=[]
    paperInfo2=[]
    start_url='http://search.cnki.com.cn'+'/Search.aspx?q='+quote(quaryWord)+'&rank=relevant=&cluster=all&val='
    for i in range(pageNum):
        hrefs=[]
        quary_url=start_url+'&p='+str(i*15)
        getURLList(quary_url,hrefs)
        getPaperInfo2(quary_url,paperInfo2)
        getPaperInfo(paperInfo,hrefs)

    data=getFormatDataset(paperInfo,paperInfo2)
    data.to_excel('E:/cnki.xlsx')    #把数据保存为excel文件
    
main()    